# 6. Enums and Pattern Matching

Enums give you a way of saying a value is one of a possible set of values

In [ ]:
{
    enum IpAddrKind {
        V4,
        V6,
    }

    struct IpAddr {
        kind: IpAddrKind,
        address: String,
    }

    let home = IpAddr {
        kind: V4,
        address: String::from("127.0.0.1"),
    };

    let loopback = IpAddr {
        kind: IpAddrKind::V6,
        address: String::from("::1"),
}

In [2]:
{
    enum IpAddr {
        V4(String),
        V6(String),
    }

    let home = IpAddr::V4(String::from("127.0.0.1"));

    let loopback = IpAddr::V6(String::from("::1"));
}

()

In [3]:
{
    enum IpAddr {
        V4(u8, u8, u8, u8),
        V6(String),
    }

    let home = IpAddr::V4(127, 0, 0, 1);

    let loopback = IpAddr::V6(String::from("::1"));
}

()

In [5]:
{
    struct Ipv4Addr {
        // --snip--
    }
    
    struct Ipv6Addr {
        // --snip--
    }
    
    enum IpAddr {
        V4(Ipv4Addr),
        V6(Ipv6Addr),
    }
}

()

The below has a wide variety of types embedded in its variants

- `Quit` has no data associated with it at all.
- `Move` has named fields, like a struct does
- `Write` includes a single String
- `ChangeColor` includes three i32 values.

It's similar `struct` definition will be like the below demo.es.

In [6]:
{
    enum Message {
        Quit,
        Move { x: i32, y: i32 },
        Write(String),
        ChangeColor(i32, i32, i32),
    }

    struct QuitMessage; // unit struct
    struct MoveMessage {
        x: i32,
        y: i32
    }
    struct WriteMessage(String); // tuple struct
    struct ChangeColorMessage(i32, i32, i32); // tuple struct 
}

()

There is one more similarity between enums and structs: just as we’re able to define methods on structs using impl, we’re also able to define methods on enums

*Note*: enum is more like union here, corresponding method needs to decide the type of `self` first.

In [11]:
{
    enum Message {
        Quit,
        Move { x: i32, y: i32 },
        Write(String),
        ChangeColor(i32, i32, i32),
    }

    impl Message {
        fn call(&self) {
            match self {
                Message::Write(m) => println!("Write: {}", m),
                _ => println!("Not Write type"),
            
            }
        }
    }

    let m = Message::Write(String::from("hello"));
    m.call();
}

Write: hello


()

## The Option Enum and Its Advantages Over Null Values

The `Option` type encodes the very common scenario in which a value could **be something or it could be nothing**.

Rust doesn’t have the null feature that many other languages have. The problem with null values is that if you try to use a null value as a not-null value, you’ll get an error of some kind.

However, the concept that null is trying to express is still a useful one: a null is a value that is currently invalid or absent for some reason.

Rust does not have nulls, but it does have an enum that can encode the concept of a value being present or absent

In [ ]:
{
    // demo of Option definition
    enum OptionX<T> {
        None,
        Some(T),
    }
}

In [13]:
{
    let some_num = Some(5);
    let some_char = Some(b'e');

    let absent_number: Option<i32> = None;
}

()

In short, because `Option<T>` and `T` (where T can be any type) are different types, the compiler won’t let us use an `Option<T>` value as if it were definitely a valid value. For example, this code won’t compile, because it’s trying to add an i8 to an `Option<i8>`.

In other words, you have to convert an `Option<T>` to a T before you can perform T operations with it. Generally, this helps catch one of the most common issues with null: assuming that something isn’t null when it actually is.

In [14]:
{
    let x: i8 = 5;
    let y: Option<i8> = Some(5);

    let sum = x + y;
}

Error: cannot add `Option<i8>` to `i8`

# 6.2. The match Control Flow Construct

In [18]:
{
    enum Coin {
        Penny,
        Nickel,
        Dime,
        Quarter,
    }

    fn value_in_cents(coin: &Coin) -> u8 {
        match coin {
            Coin::Penny => {
                println!("Lucky Penny.");
                1
            },
            Coin::Nickel => 5,
            Coin::Dime => 10,
            Coin::Quarter => 25,
        }
    }

    let a_dime = Coin::Dime;
    println!("A dime = {} cents", value_in_cents(&a_dime));

    let a_penny = Coin::Penny;
    println!("A penny = {} cents", value_in_cents(&a_penny));
}

A dime = 10 cents
Lucky Penny.
A penny = 1 cents


()

## Patterns That Bind to Values

In [20]:
{
    enum Coin {
        Penny,
        Nickel,
        Dime,
        Quarter(UsState),
    }

    #[derive(Debug)]
    enum UsState {
        Alabama,
        Alaska,
    }

    fn value_in_cents(coin: &Coin) -> u8 {
        match coin {
            Coin::Penny => {
                println!("Lucky Penny.");
                1
            },
            Coin::Nickel => 5,
            Coin::Dime => 10,
            Coin::Quarter(state) => {
                println!("state quarter from {:?}", state);
                25
            },
        }
    }

    let a_quarter = Coin::Quarter(UsState::Alaska);
    println!("A quarter = {} cents", value_in_cents(&a_quarter));

    // let a_penny = Coin::Penny;
    // println!("A penny = {} cents", value_in_cents(&a_penny));
}

state quarter from Alaska
A quarter = 25 cents


()

## Matching with `Option<T>`

In [21]:
{
    fn plus_one_opt(x: Option<i32>) -> Option<i32> {
        match x {
            None => None,
            Some(i) => Some(i + 1),
        }
    }

    let five = Some(5);
    let six = plus_one_opt(five);
    let none = plus_one_opt(None);
}

()

## Catch-all Patterns and the `_`Placeholder

In [22]:
{
    let dice_roll = 9;
    match dice_roll {
        3 => add_fancy_hat(),
        7 => remove_fancy_hat(),
        _ => (), // The catch-all branch
    }

    fn add_fancy_hat() {}
    fn remove_fancy_hat() {}
}

()

# 6.3.3 Concise Control Flow with if let

The `if let` syntax lets you combine if and let into a less verbose way to handle values that match one pattern while ignoring the rest.

`if let [option] = var { expression }`

In [23]:
{
    let config_max = Some(3u8);
    match config_max {
        Some(max) => println!("The maximum is configured to be {}", max),
        _ => (),
    }
}

The maximum is configured to be 3


()

In [24]:
{
    let config_max = Some(3u8);
    if let Some(max) = config_max {
        println!("The maximum is configured to be {}", max);
    }
}

The maximum is configured to be 3


()

In [25]:
{
    enum Coin {
        Penny,
        Nickel,
        Dime,
        Quarter(UsState),
    }

    #[derive(Debug)]
    enum UsState {
        Alabama,
        Alaska,
    }
    
    let mut count = 0;
    let coin = Coin::Quarter(UsState::Alaska);

    match coin {
        Coin::Quarter(state) => println!("State quarter from {:?}!", state),
        _ => count += 1,
    }
}

State quarter from Alaska!


()

In [27]:
{
    enum Coin {
        Penny,
        Nickel,
        Dime,
        Quarter(UsState),
    }

    #[derive(Debug)]
    enum UsState {
        Alabama,
        Alaska,
    }
    
    let mut count = 0;
    let coin = Coin::Quarter(UsState::Alaska);

    if let Coin::Quarter(state) = coin {
        println!("State quarter from {:?}!", state);
    } else {
        count += 1
    }
}

State quarter from Alaska!


()